# KNN Model with User Based Filtering
A well-known model to generate relationships and recommendations. This one is based on users which has made simliar ratings to the movies.

# 

In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
# Cargar los datos
movies = pd.read_csv('../data/movie.csv')
ratings = pd.read_csv('../data/rating.csv')

In [3]:
# Filtrar usuarios con ID entre 1 y 1000
filtered_ratings = ratings[ratings['userId'].between(1, 20000)]

In [4]:
scaler = MinMaxScaler()

# Aplicar normalización a los datos filtrados
filtered_ratings['rating'] = scaler.fit_transform(filtered_ratings[['rating']])

# Verificar el resultado
filtered_ratings.head()

C:\Users\Cheo\AppData\Local\Temp\ipykernel_6168\3195973474.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ratings['rating'] = scaler.fit_transform(filtered_ratings[['rating']])


,userId,movieId,rating,timestamp
0,1,2,0.666667,2005-04-02 23:53:47
1,1,29,0.666667,2005-04-02 23:31:16
2,1,32,0.666667,2005-04-02 23:33:39
3,1,47,0.666667,2005-04-02 23:32:07
4,1,50,0.666667,2005-04-02 23:29:40


In [5]:
# Unir los datos con el archivo movies para obtener los títulos
ratings_with_titles = filtered_ratings.merge(movies, on='movieId')

# Crear matriz de usuario-película con títulos
user_movie_matrix = ratings_with_titles.pivot_table(
    index='userId', columns='title', values='rating'
).fillna(0)

In [6]:
# Entrenar el modelo KNN
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(user_movie_matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
joblib.dump(knn, 'knn_model.pkl')
user_movie_matrix.to_pickle('user_movie_matrix.pkl')

In [ ]:
# # Función para obtener recomendaciones con títulos
# def get_recommendations(user_id, k=5, top_n=20):
#     user_index = user_id - 1  # Ajuste si el índice comienza en 0
#     distances, indices = knn.kneighbors([user_movie_matrix.iloc[user_index]], n_neighbors=k)

#     similar_users = indices.flatten()  # Usuarios similares
#     recommended_movies = []

#     # Recorrer usuarios similares para recolectar sus películas mejor calificadas
#     for sim_user in similar_users:
#         sim_user_movies = user_movie_matrix.iloc[sim_user]
#         top_movies = sim_user_movies[sim_user_movies > 0].sort_values(ascending=False).index.tolist()
#         recommended_movies.extend(top_movies)

#     # Filtrar duplicados y películas vistas por el usuario
#     recommended_movies = list(dict.fromkeys(recommended_movies))  # Eliminar duplicados
#     user_movies = user_movie_matrix.iloc[user_index][user_movie_matrix.iloc[user_index] > 0].index.tolist()
#     recommended_movies = [movie for movie in recommended_movies if movie not in user_movies]

#     # Retornar las primeras 'top_n' recomendaciones o mensaje alternativo
#     return recommended_movies[:top_n] if recommended_movies else "No se encontraron recomendaciones."

In [ ]:
# # Ejemplo de uso
# user_id = 100  # Cambia esto por el ID del usuario que quieras analizar
# recommendations = get_recommendations(user_id, k=4, top_n=20)
# print("Películas recomendadas:", recommendations)
# print("Número de recomendaciones:", len(recommendations))